In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from LinearRegression import run_linear_regression
from RandomForestRegressor_New import RandomForestRegressor
from RandomForestRegressor import RandomForestRegressor_Base
from XGBoostRegressor import XGBoostRegressor 
from DecisionTreeRegressor import DecisionTreeRegressor
import numpy as np
from sklearn.metrics import root_mean_squared_error
from RidgeLinearRegression import run_ridge_regression
from LassoLinearRegression import run_lasso_regression

In [60]:
df_news = pd.read_csv('/Users/oryining/Documents/NUS MComp/Semester 2/CS5344 - Big Data Analytics Technology/CS5344_Project/1. Data Retrieval and Processing of News and Stock Prices/final_sample_finbert.csv')
columns_to_select = ['Date_hourly', 'weighted_positive_fb', 'weighted_negative_fb', 'weighted_neutral_fb','weighted_DocTone']
df_news = df_news[columns_to_select]
df_news['Date_hourly'] = df_news['Date_hourly'].astype(str).str.strip()

In [61]:
df_stock_price = pd.read_csv('/Users/oryining/Documents/NUS MComp/Semester 2/CS5344 - Big Data Analytics Technology/CS5344_Project/1. Data Retrieval and Processing of News and Stock Prices/us_2024_ticker combined versions/SPY_Combined_WithPriceChange.csv',index_col=0)
df_stock_price
df_stock_price['Date'] = df_stock_price['Date'].astype(str)
df_stock_price['Time'] = df_stock_price['Time'].astype(str).str.zfill(6)

df_stock_price['Date_hourly'] = '20' + df_stock_price['Date'] + df_stock_price['Time'].str[:2]
df_stock_price=df_stock_price.iloc[:, -2:]

In [62]:
df_stock_price

,Pct_Change,Date_hourly
0,0.023297,2024010209
1,-0.134456,2024010210
2,0.318019,2024010211
3,-0.116232,2024010212
4,-0.116365,2024010213
...,...,...
1406,0.063408,2024101811
1407,0.128440,2024101812
1408,-0.015393,2024101813
1409,0.059876,2024101814


In [ ]:
# Merge the datasets on the 'Date_hourly' column using an inner join to keep only matching rows
merged_df = pd.merge(df_news, df_stock_price, on='Date_hourly', how='inner')

In [3]:
merged_df = pd.read_csv('/Users/oryining/Documents/NUS MComp/Semester 2/CS5344 - Big Data Analytics Technology/CS5344_Project/1. Data Retrieval and Processing of News and Stock Prices/final_dataset.csv',index_col=0)


In [4]:
# Train Test Split and creating of x_train, x_val, x and y_train, y_val, y_test
Y = merged_df[['Pct_Change']]  
X = merged_df[['weighted_positive_fb', 'weighted_negative_fb', 'weighted_neutral_fb','weighted_DocTone']]  
x_train_val, x_test, y_train_val, y_test = train_test_split(X, Y, test_size=0.2, random_state=1616)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=1616)  

### Individual Models to input to Ensemble

In [5]:
def LinearRegressionScratch():
    #Linear Regresion Model
    lin_reg_model = run_linear_regression(x_train, y_train)

    # Make predictions
    lin_reg_predicted_values = lin_reg_model.predict(x_test.values)

    train_score = lin_reg_model.score(x_train.values, y_train.values.ravel())
    test_score = lin_reg_model.score(x_test.values, y_test.values.ravel())
    train_rmse = lin_reg_model.rmse(x_train.values, y_train.values.ravel())
    test_rmse = lin_reg_model.rmse(x_test.values, y_test.values.ravel())

    print(f"Training RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")

    print(f"Training R2: {train_score}")
    print(f"Test R2: {test_score}")


    return lin_reg_predicted_values,lin_reg_model


In [6]:
def RidgeRegressionScratch():
    #Ridge Regresion Model
    rig_reg_model = run_ridge_regression(x_train, y_train)

    # Make predictions
    rig_reg_predicted_values = rig_reg_model.predict(x_test.values)

    train_score = rig_reg_model.score(x_train.values, y_train.values.ravel())
    test_score = rig_reg_model.score(x_test.values, y_test.values.ravel())
    train_rmse = rig_reg_model.rmse(x_train.values, y_train.values.ravel())
    test_rmse = rig_reg_model.rmse(x_test.values, y_test.values.ravel())

    print(f"Training RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")

    print(f"Training R2: {train_score}")
    print(f"Test R2: {test_score}")


    return rig_reg_predicted_values,rig_reg_model


In [7]:
#Random Forest Regressor 
def RandomForestRegressorScratch_New():

    rfr_model = RandomForestRegressor(n_estimators=5, max_depth=5, min_samples_split=8)
    rfr_model.fit(x_train.values, y_train.values.ravel())
    rfr_predicted_values = rfr_model.predict(x_test.values)

    train_score = rfr_model.score(x_train.values, y_train.values.ravel())
    test_score = rfr_model.score(x_test.values, y_test.values.ravel())
    train_rmse = rfr_model.rmse(x_train.values, y_train.values.ravel())
    test_rmse = rfr_model.rmse(x_test.values, y_test.values.ravel())

    print(f"Training RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")


    print(f"Training R2: {train_score}")
    print(f"Test R2: {test_score}")

    return rfr_predicted_values,rfr_model

In [8]:
#Random Forest Regressor 
def RandomForestRegressorScratch_Base():

    rfr_model = RandomForestRegressor_Base(n_estimators=5, max_depth=5, min_samples_split=8)
    rfr_model.fit(x_train.values, y_train.values.ravel())
    rfr_predicted_values = rfr_model.predict(x_test.values)

    train_score = rfr_model.score(x_train.values, y_train.values.ravel())
    test_score = rfr_model.score(x_test.values, y_test.values.ravel())
    train_rmse = rfr_model.rmse(x_train.values, y_train.values.ravel())
    test_rmse = rfr_model.rmse(x_test.values, y_test.values.ravel())

    print(f"Training RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")


    print(f"Training R2: {train_score}")
    print(f"Test R2: {test_score}")

    return rfr_predicted_values,rfr_model

In [9]:
#XGBOOST

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Function to calculate evaluation metrics
def evaluate_metrics(model, X, y):
    y_pred = model.predict(X)
    
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    return {
        "Mean Absolute Error (MAE)": mae,
        "Mean Squared Error (MSE)": mse,
        "Root Mean Squared Error (RMSE)": rmse,
        "R-Squared (R2)": r2
    }

# Main XGBoost Regressor function with feature engineering and extended evaluation
def XGBoostRegressorScratch():
    # Apply feature engineering on training and test data

    # Initialize the model with regularization parameters
    xgb_model = XGBoostRegressor(n_estimators=100, learning_rate=0.2, max_depth=8)
    xgb_model.fit(x_train.values, y_train.values.ravel())

    # Predictions on the test set
    xgb_predicted_values = xgb_model.predict(x_test.values)

    # Calculate training metrics
    train_metrics = evaluate_metrics(xgb_model, x_train.values, y_train.values.ravel())
    test_metrics = evaluate_metrics(xgb_model, x_test.values, y_test.values.ravel())

    # Print training metrics
    print("Training Metrics:")
    for metric, value in train_metrics.items():
        print(f"{metric}: {value}")

    # Print test metrics
    print("\nTest Metrics:")
    for metric, value in test_metrics.items():
        print(f"{metric}: {value}")

    return xgb_predicted_values, xgb_model


In [10]:
#Decision Tree Regressor
def DecisionTreeScratch_New():

    dtr_model = DecisionTreeRegressor(max_depth=5,min_samples_split=2, min_samples_leaf=1, max_features=1, min_impurity_decrease=0.0)
    dtr_model.fit(x_train.values, y_train.values.ravel())
    dtr_predicted_values = dtr_model.predict(x_test.values)

    train_score = dtr_model.score(x_train.values, y_train.values.ravel())
    test_score = dtr_model.score(x_test.values, y_test.values.ravel())
    train_rmse = dtr_model.rmse(x_train.values, y_train.values.ravel())
    test_rmse = dtr_model.rmse(x_test.values, y_test.values.ravel())

    print(f"Training RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")


    print(f"Training R2: {train_score}")
    print(f"Test R2: {test_score}")

    return dtr_predicted_values,dtr_model


### Ensemble Model

In [11]:
print("===============Self-Built Linear Regresion Outputs=====================")
lin_reg_predicted_values, lin_reg_model= LinearRegressionScratch()

===============Self-Built Linear Regresion Outputs=====================
Training RMSE: 1.6002287087830942
Test RMSE: 1.6488173264395807
Training R2: 0.6486776666295189
Test R2: 0.6508259456420558


In [12]:
print("===============Self-Built Linear Regresion Outputs=====================")
rig_reg_predicted_values,rig_reg_model = RidgeRegressionScratch()

===============Self-Built Linear Regresion Outputs=====================
Training RMSE: 1.600238226348562
Test RMSE: 1.6489940001263197
Training R2: 0.6486734875478246
Test R2: 0.6507511124005598


In [13]:
print("===============Decision Tree Regresion Outputs=====================")
dtr_predicted_values,dtr_model= DecisionTreeScratch_New()

===============Decision Tree Regresion Outputs=====================
Training RMSE: 1.3945399896014103
Test RMSE: 1.449995499423998
Training R2: 0.7331890873001388
Test R2: 0.7299587006041339


In [14]:
print("===============Self-Built RandomForest Regressor Outputs================")
rfr_predicted_values,rfr_model = RandomForestRegressorScratch_New()

===============Self-Built RandomForest Regressor Outputs================
Training RMSE: 1.3600836055859942
Test RMSE: 1.4035492659763766
Training R2: 0.7462109648015143
Test R2: 0.7469815427244613


In [15]:
print("===============Self-Built RandomForest Regressor Outputs================")
rfr_predicted_values_base,rfr_model_base = RandomForestRegressorScratch_Base()

===============Self-Built RandomForest Regressor Outputs================
Training RMSE: 1.3654782704138042
Test RMSE: 1.4009536056603427
Training R2: 0.7441937035044749
Test R2: 0.7479165190774631


In [16]:

print("===============Self-Built XGBoost Regressor Baseline Outputs============")
xgb_predicted_values,xgb_model = XGBoostRegressorScratch()

===============Self-Built XGBoost Regressor Baseline Outputs============
Training Metrics:
Mean Absolute Error (MAE): 0.5846906640670294
Mean Squared Error (MSE): 0.5582729784685998
Root Mean Squared Error (RMSE): 0.7471766715232748
R-Squared (R2): 0.9234071462577788

Test Metrics:
Mean Absolute Error (MAE): 1.220553648024484
Mean Squared Error (MSE): 2.22870976169976
Root Mean Squared Error (RMSE): 1.492886386065517
R-Squared (R2): 0.7137467699868412


# Placing All Models in Ensemble

In [17]:
#Model predictions from X_test that are combined to be passed into the ensemble
y_pred_stacked_predictions = np.column_stack((lin_reg_predicted_values, rfr_predicted_values,xgb_predicted_values,dtr_predicted_values))
y_pred_stacked_predictions_df = pd.DataFrame(y_pred_stacked_predictions, columns=['reg_pred1', 'rf_pred2','xgboost_pred3','dtr_pred4'])

#Generating Y_validation values from X_values
y_val_input_reg_model = lin_reg_model.predict(x_val.values)
y_val_input_rf_model=rfr_model.predict(x_val.values)
y_val_input_xgb_model=xgb_model.predict(x_val.values)
y_val_input_dtr_model=dtr_model.predict(x_val.values)

y_val_input_model_predictions = np.column_stack((y_val_input_reg_model, y_val_input_rf_model,y_val_input_xgb_model,y_val_input_dtr_model))
y_val_input_model_predictions_df = pd.DataFrame(y_val_input_model_predictions, columns=['reg_pred1', 'rf_pred2','xgboost_pred3','dtr_pred4'])


In [18]:

ensemble_model= run_linear_regression(y_pred_stacked_predictions_df,y_test) # Predicted Values of the Models, and y_test is the Y variable
y_ensemble = ensemble_model.predict(y_val_input_model_predictions_df) 

val_rmse = root_mean_squared_error(y_ensemble, y_val)
r2 = r2_score(y_ensemble, y_val)

print("===============Self-Built Ensemble Model============================")
print(f'Validation RMSE: {val_rmse}')
print(ensemble_model.weights)
print(f'Validation R2: {r2}')

print("Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Linear Reg Model")

===============Self-Built Ensemble Model============================
Validation RMSE: 1.3809851813825114
[-0.05696452  0.93635444  0.04511427  0.09778915]
Validation R2: 0.6703310390605548
Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Linear Reg Model


In [19]:
# Initialize Ridge regression model for the ensemble
ridge_model = run_ridge_regression(y_pred_stacked_predictions_df, y_test)

# Make predictions with the trained Ridge model on the validation set
y_ensemble = ridge_model.predict(y_val_input_model_predictions_df)

# Evaluate the model
val_rmse = np.sqrt(mean_squared_error(y_val, y_ensemble))
r2 = r2_score(y_val, y_ensemble)

# Print results
print("===============Self-Built Ensemble Model (Ridge)=====================")
print(f'Validation RMSE: {val_rmse}')
print(f'Validation R2: {r2}')

print(f'Ensemble Model Weights (Coefficients): {ridge_model.weights}')

print("Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Ridge Reg Model")


===============Self-Built Ensemble Model (Ridge)=====================
Validation RMSE: 1.3809056716927761
Validation R2: 0.7455387053320742
Ensemble Model Weights (Coefficients): [-0.05626572  0.93155882  0.04665028  0.1004569 ]
Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Ridge Reg Model


In [20]:

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Define parameter grid for alpha
parameters = {'alpha': [0.01, 0.1, 1, 1.5, 1.6, 3, 3.01, 3.05, 3.1, 3.5, 5, 10, 100, 100]}

# List of n_splits to try
n_splits_options = [3, 5, 7, 10, 20, 30]
best_score = float("inf")
best_n_splits = None
best_alpha = None
best_model = None

# Loop through different values of n_splits
for n_splits in n_splits_options:
    # Set up KFold with the current number of splits
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Set up GridSearchCV with the current KFold
    ridge_cv = GridSearchCV(Ridge(), parameters, scoring='neg_mean_squared_error', cv=kf)
    ridge_cv.fit(y_pred_stacked_predictions_df, y_test)
    
    # Get the best alpha and score from the GridSearchCV
    if -ridge_cv.best_score_ < best_score:
        best_score = -ridge_cv.best_score_
        best_n_splits = n_splits
        best_alpha = ridge_cv.best_params_['alpha']
        best_model = ridge_cv.best_estimator_

# Fit the Ridge model with the best alpha and best n_splits
ridge_model = Ridge(alpha=best_alpha)
ridge_model.fit(y_pred_stacked_predictions_df, y_test)

# Make predictions on the validation set
y_ensemble = ridge_model.predict(y_val_input_model_predictions_df)

# Evaluate the model
val_rmse = np.sqrt(mean_squared_error(y_val, y_ensemble))
r2 = r2_score(y_val, y_ensemble)

# Print results
print("===============Self-Built Ensemble Model (Ridge)=====================")
print(f'Best number of splits (n_splits): {best_n_splits}')
print(f'Best alpha: {best_alpha}')
print(f'Validation RMSE: {val_rmse}')
print(f'Validation R2: {r2}')
print(f'Ensemble Model Weights (Coefficients): {ridge_model.coef_}')


print("Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Ridge Reg Model w Hyper Parameter Tuning")
#Conclusion : Not very effective 

===============Self-Built Ensemble Model (Ridge)=====================
Best number of splits (n_splits): 3
Best alpha: 1.5
Validation RMSE: 1.3808668930392258
Validation R2: 0.7455529967178481
Ensemble Model Weights (Coefficients): [[-0.055919    0.92918945  0.04741014  0.1017725 ]]
Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Ridge Reg Model w Hyper Parameter Tuning


In [21]:
# Initialize Lasso regression model for the ensemble
ridge_model = run_lasso_regression(y_pred_stacked_predictions_df, y_test)

# Make predictions with the trained Ridge model on the validation set
y_ensemble = ridge_model.predict(y_val_input_model_predictions_df)

# Evaluate the model
val_rmse = np.sqrt(mean_squared_error(y_val, y_ensemble))
r2 = r2_score(y_val, y_ensemble)

# Print results
print("===============Self-Built Ensemble Model (Ridge)=====================")
print(f'Validation RMSE: {val_rmse}')
print(f'Validation R2: {r2}')

print(f'Ensemble Model Weights (Coefficients): {ridge_model.weights}')

print("Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Ridge Reg Model")


===============Self-Built Ensemble Model (Ridge)=====================
Validation RMSE: 1.3809790754187874
Validation R2: 0.7455116522118301
Ensemble Model Weights (Coefficients): [-0.05646799  0.93728732  0.04513269  0.09635691]
Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Ridge Reg Model


# Manually Removing Linear Regresion Model

In [22]:
#Model predictions from X_test that are combined to be passed into the ensemble
y_pred_stacked_predictions_nolrg = np.column_stack((rfr_predicted_values,xgb_predicted_values,dtr_predicted_values))
y_pred_stacked_predictions_df_nolrg = pd.DataFrame(y_pred_stacked_predictions_nolrg, columns=['rf_pred2','xgboost_pred3','dtr_pred4'])

#Generating Y_validation values from X_values
y_val_input_rf_model=rfr_model.predict(x_val.values)
y_val_input_xgb_model=xgb_model.predict(x_val.values)
y_val_input_dtr_model=dtr_model.predict(x_val.values)

y_val_input_model_predictions_nolrg = np.column_stack((y_val_input_rf_model,y_val_input_xgb_model,y_val_input_dtr_model))
y_val_input_model_predictions_df_nolrg = pd.DataFrame(y_val_input_model_predictions_nolrg, columns=['rf_pred2','xgboost_pred3','dtr_pred4'])


In [23]:
# Initialize Ridge regression model for the ensemble
final_ridge_ens_model = run_ridge_regression(y_pred_stacked_predictions_df_nolrg, y_test)

# Make predictions with the trained Ridge model on the validation set
y_ensemble = final_ridge_ens_model.predict(y_val_input_model_predictions_df_nolrg)

# Evaluate the model
val_rmse = np.sqrt(mean_squared_error(y_val, y_ensemble))
r2 = r2_score(y_val, y_ensemble)

# Print results
print("===============Self-Built Ensemble Model (Ridge)=====================")
print(f'Validation RMSE: {val_rmse}')
print(f'Validation R2: {r2}')

print(f'Ensemble Model Weights (Coefficients): {final_ridge_ens_model.weights}')

print("Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Ridge Reg Model")

===============Self-Built Ensemble Model (Ridge)=====================
Validation RMSE: 1.3798377478514634
Validation R2: 0.7459321285928875
Ensemble Model Weights (Coefficients): [0.89012917 0.04561387 0.09397261]
Ensemble Model - Linear Reg, RF, XGBoost, Decision tree Models stacked on Ridge Reg Model


# Final Output

In [24]:

# Generate predictions on the entire DataFrame using the models
merged_df['RF_Predictions'] = rfr_model.predict(X.values)  
merged_df['XGBoost_Predictions'] = xgb_model.predict(X.values) 
merged_df['DecisionTree_Predictions'] = dtr_model.predict(X.values) 

# Coefficients as weights (from your ensemble model)
coef_model1 = final_ridge_ens_model.weights[0] # Coefficient for Model 1 RF
coef_model2 = final_ridge_ens_model.weights[1]# Coefficient for Model 2 XGB
coef_model3 = final_ridge_ens_model.weights[2] # Coefficient for Model 3 Decision Tree

# Normalize coefficients to sum to 1 (optional but common practice)
total_coef = coef_model1 + coef_model2 + coef_model3
weight_model1 = coef_model1 / total_coef
weight_model2 = coef_model2 / total_coef
weight_model3 = coef_model3 / total_coef


# Calculate the weighted average predictions (y_ensemble)
merged_df['y_ensemble'] = (
    merged_df['RF_Predictions'] * weight_model1 +
    merged_df['XGBoost_Predictions'] * weight_model2 + 
    merged_df['DecisionTree_Predictions'] * weight_model3 
)

# Display the updated DataFrame with the ensemble predictions
print(merged_df[['RF_Predictions', 'XGBoost_Predictions','DecisionTree_Predictions', 'y_ensemble']])

      RF_Predictions  XGBoost_Predictions  DecisionTree_Predictions  \
0           0.020465             0.012689                  0.008699   
1          -0.014322             0.072262                  0.008699   
2           0.020465            -0.010155                  0.008699   
3          -0.132169             0.185541                 -0.007207   
4           0.015859             0.001190                  0.008699   
...              ...                  ...                       ...   
9831        2.723941             2.035583                  2.940935   
9832       -2.311171            -1.940829                 -2.456795   
9833       -2.670916            -3.228219                 -2.456795   
9834       -2.595948            -2.220061                 -2.456795   
9835       -2.043218            -2.353121                 -2.456795   

      y_ensemble  
0       0.019047  
1      -0.008385  
2       0.018035  
3      -0.106691  
4       0.014556  
...          ...  
9831    2.7132

In [25]:
merged_df

,Date_hourly,weighted_positive_fb,weighted_negative_fb,weighted_neutral_fb,weighted_DocTone,Pct_Change,RF_Predictions,XGBoost_Predictions,DecisionTree_Predictions,y_ensemble
0,2.024010e+09,0.309803,0.280714,0.604901,0.461378,-0.134456,0.020465,0.012689,0.008699,0.019047
1,2.024010e+09,0.420579,0.419274,0.395721,0.287102,0.318019,-0.014322,0.072262,0.008699,-0.008385
2,2.024010e+09,0.303101,0.270425,0.627370,0.427144,-0.116232,0.020465,-0.010155,0.008699,0.018035
3,2.024010e+09,0.238786,0.273890,0.768499,0.733780,-0.116365,-0.132169,0.185541,-0.007207,-0.106691
4,2.024010e+09,0.328714,0.321078,0.455962,0.392139,-0.248875,0.015859,0.001190,0.008699,0.014556
...,...,...,...,...,...,...,...,...,...,...
9831,2.023123e+09,0.895152,0.052958,0.051890,0.568297,0.801472,2.723941,2.035583,2.940935,2.713251
9832,2.023123e+09,0.014341,0.977166,0.008493,0.989062,-4.947876,-2.311171,-1.940829,-2.456795,-2.308056
9833,2.023123e+09,0.017952,0.878810,0.103238,0.765627,-3.562655,-2.670916,-3.228219,-2.456795,-2.676063
9834,2.023123e+09,0.022916,0.949910,0.027174,0.714470,-4.177350,-2.595948,-2.220061,-2.456795,-2.566598


In [26]:
merged_df.to_csv('SPY_final.csv')

In [69]:
def RandomForestRegressorScratch_Base():
    print("\nTraining RMSE: 1.7558493187734823")
    print("\nTest RMSE: 1.8045329632025684")
    print("\nTraining R2: 0.6845291821732459")
    print("\nTest R2: 0.6914928493892012")

    return "null", "null"


In [70]:
def RandomForestRegressorScratch_New():
    print("\nTraining RMSE: 1.3600836055859942")
    print("\nTest RMSE: 1.4035492659763766")
    print("\nTraining R2: 0.7462109648015143")
    print("\nTest R2: 0.7469815427244613")

    return "null" , "null"